# 🔧 Tuning Estrategia SMA 10/50 - Cruce v2

## Descripción de la estrategia
Esta estrategia detecta cruces entre medias móviles simples:
- **SMA corto** (por defecto 10) vs **SMA largo** (por defecto 50)
- Señales de compra ("buy") se generan cuando la SMA10 cruza por encima de la SMA50.
- Señales de venta ("sell") se generan cuando la SMA10 cruza por debajo de la SMA50.

### Mejoras incluidas:
- Confirmación opcional del cruce al día siguiente
- Filtro de volatilidad basado en ATR
- Validación con estructura de vela japonesa (cuerpo)

## Parámetros configurables
- `window_corto`: int
- `window_largo`: int
- `confirmar_al_dia_siguiente`: bool
- `usar_filtro_volatilidad`: bool
- `confirmar_cuerpo_vela`: bool

## Objetivo del notebook
Realizar tuning de la estrategia mediante evaluación por grid de combinaciones, para identificar las configuraciones con mejor desempeño según un `score` definido en la simulación.

Los datos se cargan desde la ruta local `D:/trading/data/historic`.
"""

In [1]:
import os
import pandas as pd
from pathlib import Path

ruta_historicos = Path("D:/trading/data/historic")
parquets = list(ruta_historicos.glob("*.parquet"))
historicos = {}

for archivo in parquets:
    simbolo = archivo.stem
    try:
        df = pd.read_parquet(archivo)
        if "fecha" in df.columns:
            df["fecha"] = pd.to_datetime(df["fecha"])
            historicos[simbolo] = df.sort_values("fecha").reset_index(drop=True)
    except Exception as e:
        print(f"Error al cargar {archivo.name}: {e}")

print(f"Se cargaron {len(historicos)} símbolos.")

Se cargaron 48 símbolos.


In [2]:
import itertools

param_grid = {
    "window_corto": [8, 10, 12],
    "window_largo": [45, 50, 55],
    "confirmar_al_dia_siguiente": [False, True],
    "usar_filtro_volatilidad": [False, True],
    "confirmar_cuerpo_vela": [False, True],
}

claves = list(param_grid.keys())
combinaciones = list(itertools.product(*param_grid.values()))
print(f"Total de combinaciones a evaluar: {len(combinaciones)}")

Total de combinaciones a evaluar: 72


In [3]:
import sys
sys.path.append("D:/trading")
from my_modules.estrategias.v3 import sma_10_50_cruce_v3

def simular_combinacion(args):
    combinacion, historicos = args
    params = dict(zip(claves, combinacion))

    resultados = []
    for simbolo, df in historicos.items():
        try:
            df_out = sma_10_50_cruce_v2.generar_senales(df.copy(), **params)
            df_out["simbolo"] = simbolo
            resultados.append(df_out)
        except Exception as e:
            print(f"Error {simbolo}: {e}")

    if not resultados:
        return {"params": params, "score": -999, "n": 0}

    df_all = pd.concat(resultados)
    n_signals = (df_all.signal != "hold").sum()
    buy_ratio = (df_all.signal == "buy").sum() / max(n_signals, 1)
    score = round(n_signals * (1 + buy_ratio), 2)

    return {"params": params, "score": score, "n": n_signals}

In [4]:
from joblib import Parallel, delayed
from tqdm import tqdm

def ejecutar_tuning(historicos, combinaciones):
    def safe_simulacion(comb):
        try:
            return simular_combinacion((comb, historicos))
        except Exception as e:
            print(f"Error en combinación {comb}: {e}")
            return {"params": dict(zip(claves, comb)), "score": -999, "n": 0}

    resultados = Parallel(n_jobs=-1, backend="loky")(
        delayed(safe_simulacion)(comb) for comb in tqdm(combinaciones)
    )
    return resultados

In [5]:
df_resultados = pd.DataFrame(resultados)
df_resultados = df_resultados.sort_values("score", ascending=False).reset_index(drop=True)
df_resultados["params_str"] = df_resultados["params"].astype(str)
df_resultados[["score", "n", "params_str"]].head(10)

NameError: name 'resultados' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(df_resultados["score"].values, marker="o")
plt.title("Distribución de Score por Configuración")
plt.xlabel("Ranking")
plt.ylabel("Score")
plt.grid(True)
plt.tight_layout()
plt.show()